## Программа

- Подоготовить тексты
- Скачать готовую модель
- Подготовка окружения для работы
- Дообучение модели
- Проверка корректности

In [1]:
import sqlite3
import pandas as pd
import re # модуль регулярных выражений для работы с текстом

In [2]:
def replace_all(text, to_replace):
    for i in to_replace:
        text = text.replace(i, ' ')
    return text

In [3]:
con = sqlite3.connect('songs.db')

cur = con.cursor()
res = cur.execute('Select * from text_songs') # вытаскиваем все значения из таблицы
result = res.fetchall()

In [4]:
result[0]

('Григорий Лепс',
 'Где-то там  За окном  Ходит зима  Сеет снег  Белый снег  Ночью и днем  И меня тишиной  Сводит с ума  И опять не уснуть  В доме пустом    Тихо  Саваном белым  Вьюга  Дом мой укрой  Где-то ты засыпаешь  Где-то, но не со мной    Где-то там  В тишине  Ходит февраль  И ему  Как и мне  Сон не найти  Где-то там  Вдалеке  Никогда ты не поймешь  Где-то там  Не со мной  Так и не узнаешь ты    Кружит  Белая вьюга  Тихо  Ходит зима  Слышишь  Как замерзаю  Снова  Я без тебя    Эяяяяау    Тихо  Саваном белым  Вьюга  Дом мой укрой  Где-то ты засыпаешь  Где-то, но не со мной    Кружит  Белая вьюга  Тихо  Ходит зима  Слышишь  Как замерзаю  Снова  Я без тебя',
 'https://text-you.ru//rus_text_pesni/4559-grigorijj-leps-vjuga.html')

In [5]:
text_songs = [i[1] for i in result] # вытаскиваем только тексты песен и записываем в отдельный список
text_songs[0]

'Где-то там  За окном  Ходит зима  Сеет снег  Белый снег  Ночью и днем  И меня тишиной  Сводит с ума  И опять не уснуть  В доме пустом    Тихо  Саваном белым  Вьюга  Дом мой укрой  Где-то ты засыпаешь  Где-то, но не со мной    Где-то там  В тишине  Ходит февраль  И ему  Как и мне  Сон не найти  Где-то там  Вдалеке  Никогда ты не поймешь  Где-то там  Не со мной  Так и не узнаешь ты    Кружит  Белая вьюга  Тихо  Ходит зима  Слышишь  Как замерзаю  Снова  Я без тебя    Эяяяяау    Тихо  Саваном белым  Вьюга  Дом мой укрой  Где-то ты засыпаешь  Где-то, но не со мной    Кружит  Белая вьюга  Тихо  Ходит зима  Слышишь  Как замерзаю  Снова  Я без тебя'

In [7]:
text_prep_all = []
to_replace = ['григорий', 'лепс', '!', '?', '.', ',', ':', '-', '  ', '[', ']', '))', '"', '(', ')', '—', '  '] # мусор для очистки



for i in text_songs:
    text_prep = [replace_all(a, to_replace) for a in i]
    text_prep = [a.rstrip().rstrip() for a in text_prep]
    text_prep = re.findall('[А-Я][^А-Я]*', i) # Делит текст на элементы списка, где заглавная буква - триггер разрыва
    text_prep = [a.lower() for a in text_prep] # все в нижний регистр
    text_prep = [a.lstrip() for a in text_prep]
    clean_text = []
    for idx, val in enumerate(text_prep): #крутой метод обработки списка, как словаря. Здесь это способ поитерироваться по элементам и пересобрать нужные в новом списке
        if text_prep[idx] in [' ', '']:
            pass
        else:
            clean_text.append(val)
            
    text_prep = []
    for val in clean_text:
        if len(val) <= 10: #убираем короткие фразы
            pass
        else:
            text_prep.append(val)
    
    text_prep_unique = list(set(text_prep)) #Убираем дубликаты
    text_prep_str = '\n'.join(text_prep_unique) #склеиваем все обратно, добавляем /n для правильного отображения текста песни
    text_prep_all.append(text_prep_str) # собираем все тексты в список
    

In [8]:
len(text_prep_all)

54

In [13]:
print (text_prep_all[0])

белый снег  
где-то ты засыпаешь  
я без тебя    
так и не узнаешь ты    
в доме пустом    
как замерзаю  
как и мне  
сеет снег  
сводит с ума  
ночью и днем  
белая вьюга  
дом мой укрой  
сон не найти  
саваном белым  
где-то там  
ходит зима  
эяяяяау    
никогда ты не поймешь  
ходит февраль  
и меня тишиной  
не со мной  
где-то, но не со мной    
и опять не уснуть  


In [12]:
#Собираем все в df

df_prepped = pd.DataFrame()
df_prepped['text'] = text_prep_all * 10 # Это для увеличения учебного материала для модели
df_prepped['artist'] = 'Григорий Лепс'
df_prepped = df_prepped[['artist', 'text']]
df_prepped.to_csv('lyrics-generator/leps_text_v3.csv', index=False)

# именно в таком формате данные готова кушать наша модель
print(df_prepped.shape)
df_prepped.head()

(540, 2)


,artist,text
0,Григорий Лепс,белый снег \nгде-то ты засыпаешь \nя без теб...
1,Григорий Лепс,на пути моем встает - \nзазвенели в свой чере...
2,Григорий Лепс,"видела звезда, как отцы и деды делали победы, ..."
3,Григорий Лепс,"забери меня в свой плен. \nможет, даже, не по..."
4,Григорий Лепс,"живу себе, себя воображая, \nискал поддержки я..."
